To generate the input file here, use the BEAST software called  TreeMarkovJumpHistoryAnalyzer (command in same folder in repo). Make sure you remove burnin in at that stage because we are assuming it has been removed for these commands

In [1]:
import csv
from collections import defaultdict
from collections import Counter
import baltic
import matplotlib.pyplot as plt
import geopandas as gpd
import tqdm
import numpy as np
import dendropy
import seaborn as sns
import geopandas as gpd
import matplotlib.patches as mpatches
import matplotlib as mpl

font = {'family' : 'Helvetica',
'weight' : 'bold',
'size' : 18}
mpl.rcParams.update({"svg.fonttype": 'none', 'text.usetex': False})

In [5]:
markov_file = ""
map_file = gpd.read_file("") #map json - I recommend gadm.org

name_of_resolution = "" #eg "state" if that's what level you're plotting at and that's what it's called in the map json

In [2]:
def parse_jumps(markov_file):

    all_tree_movements = defaultdict(list)

    with open(markov_file) as f:
        data = csv.DictReader(f)
        for l in tqdm.tqdm(data):
            tree = l['treeId']
            if l['startLocation'] != l['endLocation']: #they should all be different but just to check
                all_tree_movements[tree].append((l['startLocation'],l['endLocation'],l['time']))
                
    average_freqs, upper_freqs, lower_freqs = make_pairs(all_tree_movements)
                    
    return average_freqs, upper_freqs, lower_freqs

In [3]:
def make_pairs(all_tree_movements):

    pairs = defaultdict(list)
    
    for tree, lst in all_tree_movements.items():
        for mvmt in lst:
            pairs[tree].append((mvmt[0], mvmt[1]))
    
    tree_places_counts = defaultdict(dict)
    for tree, mvmts in (pairs.items()):
        tree_places_counts[tree] = Counter(mvmts)

    loc_freqs = defaultdict(list)
    for tree, loc_dict in (tree_places_counts.items()):
        for loc, count in loc_dict.items():
            freq = count/sum(loc_dict.values())
            loc_freqs[loc].append(freq)
            
    average_freqs = {}
    lower_freqs = {}
    upper_freqs = {}

    for pair, freq_lst in loc_freqs.items():
        average_freqs[pair] = np.mean(freq_lst)
        upper_freqs[pair] = np.percentile(freq_lst, 97.5)
        lower_freqs[pair] = np.percentile(freq_lst, 2.5)

    return average_freqs, upper_freqs, lower_freqs

In [4]:
def make_map_figure(map_file, centroid_dict, average_freqs, save_name, multiplier=50):

    fig, ax = plt.subplots(1,1, figsize=(10,10))

    map_file.plot(ax=ax, color="lightgrey", edgecolor="white", linewidth=2)

    for pair, freq in average_freqs.items():

        start = (centroid_dict[pair[0]].x, centroid_dict[pair[0]].y)
        end = (centroid_dict[pair[1]].x, centroid_dict[pair[1]].y)
        width = freq*multiplier

        kw = dict(color="slategrey",linewidth=width)
        liney = mpatches.FancyArrowPatch(start, end, arrowstyle="-",
                                 connectionstyle="arc3,rad=.5", **kw)

        ax.add_patch(liney)

    ax.axis('off')
    
    plt.savefig(f"{save_name}.png", bbox_inches="tight")
    plt.savefig(f"{save_name}.pdf", bbox_inches="tight")

In [ ]:
all_movements, upper_all, lower_all = parse_jumps(markov_file)




In [6]:
map_file['centroids'] = map_file.geometry.centroid
map_file_centroid = {}
for place, centroid in zip(map_file[name_of_resolution], map_file['centroids']):
    map_file_centroid[place.replace(" ","_")] = centroid
    
    
      

NameError: name 'map_file' is not defined

In [ ]:
make_map_figure(map_file, map_file_centroid, all_movements, "all_movements", multiplier=50) #fiddle with multiplier to make it look nice

